In [1]:
import numpy as np

def givens_rotation(a, b):
    """
    Create Givens rotation matrix for elements a and b.
    """
    if b == 0:
        c, s = 1, 0
    else:
        if abs(b) > abs(a):
            tau = -a / b
            s = 1 / np.sqrt(1 + tau ** 2)
            c = s * tau
        else:
            tau = -b / a
            c = 1 / np.sqrt(1 + tau ** 2)
            s = c * tau
    return c, s

def apply_givens_rotation(A, c, s, i, j):
    """
    Apply Givens rotation to zero out A[i,j] and A[j,i].
    """
    G = np.eye(A.shape[0])
    G[[i, j], [i, j]] = c
    G[i, j] = s
    G[j, i] = -s
    return G.T @ A @ G

def tridiagonalize(A):
    """
    Tridiagonalize symmetric matrix A using Givens rotations.
    """
    n = A.shape[0]
    for i in range(n - 1):
        for j in range(i + 2, n):
            c, s = givens_rotation(A[i+1, i], A[j, i])
            A = apply_givens_rotation(A, c, s, i + 1, j)
    return A

def sturm_sequence(A, x):
    """
    Calculate the Sturm sequence for tridiagonal matrix A at value x.
    """
    n = A.shape[0]
    p = np.zeros(n + 1)
    p[0] = 1
    p[1] = A[0, 0] - x

    for i in range(2, n + 1):
        p[i] = (A[i - 1, i - 1] - x) * p[i - 1] - A[i - 1, i - 2] ** 2 * p[i - 2]

    return p

def count_sturm_sign_changes(p):
    """
    Count the number of sign changes in the Sturm sequence.
    """
    sign_changes = 0
    for i in range(1, len(p)):
        if p[i] * p[i - 1] < 0:
            sign_changes += 1
    return sign_changes

def find_eigenvalues(A, tol=1e-10, max_iter=1000):
    """
    Find the eigenvalues of symmetric matrix A using Givens method and Sturm sequence.
    """
    n = A.shape[0]
    A = tridiagonalize(A)

    eigenvalues = []

    for i in range(n):
        lower_bound = np.min(np.diag(A)) - tol
        upper_bound = np.max(np.diag(A)) + tol

        while upper_bound - lower_bound > tol:
            midpoint = (lower_bound + upper_bound) / 2
            sturm_seq = sturm_sequence(A, midpoint)
            num_sign_changes = count_sturm_sign_changes(sturm_seq)

            if num_sign_changes > i:
                upper_bound = midpoint
            else:
                lower_bound = midpoint

        eigenvalues.append((lower_bound + upper_bound) / 2)

    return np.array(eigenvalues)

# Example usage:
A = np.array([[4, 1, 2],
              [1, 2, 3],
              [2, 3, 1]], dtype=float)
eigenvalues = find_eigenvalues(A)
print("Eigenvalues:", eigenvalues)


Eigenvalues: [-0.6         2.30838907  4.        ]
